In [4]:
import pandas as pd 
import os
from openpyxl import Workbook,load_workbook
import re
import numpy as np
from enum import Enum



path = r'./vtm7.xlsx'#这个是所有excel文件夹的路径
pathfile = './Tang'#这个比上面那个多了个/，修改的化改这两个就行
data_list=[] 
Depth_list=[]
f=open(path,'rb')
data=pd.read_excel(f)

x=data.shape


In [5]:
length=x[0]
data_list=list(data.iloc[0:length-1,0])
pattern =re.compile(r'\d+')


In [6]:
Depth_list=[]
for data_temp in data_list:
    if((data_temp.find('SplitSeries')!=-1)and(data_temp.find('POC')!=-1)and(data_temp.find('Chroma')==-1)):
        Depth_list.append(data_temp)
rows=len(Depth_list)
cols=6
rowx=0
colx=0

# # Depth_list[0]
# BlockDataStore=[[0 for rowx in range (rows)] for colx in range (cols)]
# print(len(BlockDatastore[]))



In [7]:
Depth_list

432,   0) [16x 8] SplitSeries=134177',
 'BlockStat: POC 0 @( 432,   8) [16x16] SplitSeries=134177',
 'BlockStat: POC 0 @( 432,  24) [16x 8] SplitSeries=134177',
 'BlockStat: POC 0 @( 384,  32) [16x 8] SplitSeries=2197537',
 'BlockStat: POC 0 @( 384,  40) [16x 8] SplitSeries=2197537',
 'BlockStat: POC 0 @( 400,  32) [ 4x16] SplitSeries=5343265',
 'BlockStat: POC 0 @( 404,  32) [ 8x16] SplitSeries=5343265',
 'BlockStat: POC 0 @( 412,  32) [ 4x16] SplitSeries=5343265',
 'BlockStat: POC 0 @( 384,  48) [ 8x16] SplitSeries=165921',
 'BlockStat: POC 0 @( 392,  48) [16x16] SplitSeries=165921',
 'BlockStat: POC 0 @( 408,  48) [ 8x16] SplitSeries=165921',
 'BlockStat: POC 0 @( 416,  32) [32x 8] SplitSeries=4129',
 'BlockStat: POC 0 @( 416,  40) [ 8x16] SplitSeries=167969',
 'BlockStat: POC 0 @( 424,  40) [16x16] SplitSeries=167969',
 'BlockStat: POC 0 @( 440,  40) [ 8x16] SplitSeries=167969',
 'BlockStat: POC 0 @( 416,  56) [16x 4] SplitSeries=3215393',
 'BlockStat: POC 0 @( 432,  56) [16x 4] Sp

In [8]:
BlockDataStore=np.zeros([rows,cols])
DataTempStore=[]
rowx=0
for Depth_temp in Depth_list:
    Data_temp=pattern.findall(Depth_temp)
    # if((rowx)%4!=0):
    #     BlockDataStore[rowx//4][col_temp]=np.array(Data_temp[5])
    #     rowx=rowx+1
    #     col_temp=col_temp+1
    #     continue
    # else:
    colx=0
    for Data in Data_temp:
        BlockDataStore[rowx][colx]=Data
        colx=colx+1

    rowx=rowx+1
Dim=BlockDataStore.shape
Dec_Temp=0
Bin_Temp=[]
GroundTruth=['1']
for rowx in range(Dim[0]):
    Dec_Temp=BlockDataStore[rowx][-1]
    Bin_Temp=list(bin(int(Dec_Temp)))
    Bin_Temp.reverse()
    # print(Bin_Temp)
    count=0
    Bin_Len=len(Bin_Temp)
    count_zero=0
    for count in range(Bin_Len):
        ##因列表反向，相当于从低位到高位遍历
        # print(count)
        if(count==Bin_Len-2):
            GroundTruth.append('0')
            # print(GroundTruth)
            BlockDataStore[rowx][-1]=int(("".join(GroundTruth)))##list转字符串
            GroundTruth=['1']
            break
        elif(Bin_Temp[count]=='1'):
            ##m每四个0为一个有效元素
            if((count_zero==1)or(count_zero==0)):
                continue
            elif(count_zero==4):
                #两个连续的1
                if(Bin_Temp[count+1]=='1'):##11
                    GroundTruth.append('3')
                elif((Bin_Temp[count+1]=='0')and(Bin_Temp[count+2]=='1')):##101
                    GroundTruth.append('5')
                else:#1
                    GroundTruth.append('1')
            elif(count_zero==5):
                ##10
                GroundTruth.append('2')
            elif(count_zero==6):
                #100
                GroundTruth.append('4')
            count_zero=0
            # print(GroundTruth)
        else:
            count_zero=count_zero+1
            
            

    # Bin_Temp=list(bin(int(Dec_Temp))
    # print(Bin_Temp)


In [9]:
BlockDataStore

array([[0.000e+00, 0.000e+00, 0.000e+00, 3.200e+01, 3.200e+01, 1.100e+02],
       [0.000e+00, 3.200e+01, 0.000e+00, 3.200e+01, 1.600e+01, 1.120e+03],
       [0.000e+00, 3.200e+01, 1.600e+01, 1.600e+01, 1.600e+01, 1.120e+03],
       ...,
       [4.000e+00, 8.000e+02, 4.560e+02, 1.600e+01, 8.000e+00, 1.121e+04],
       [4.000e+00, 8.160e+02, 4.560e+02, 1.600e+01, 8.000e+00, 1.121e+04],
       [4.000e+00, 8.000e+02, 4.640e+02, 3.200e+01, 1.600e+01, 1.120e+03]])

In [10]:
filepath_write = './GroundTruthoutput'+str(1)+'.xlsx'
filepath_write

'./GroundTruthoutput1.xlsx'

In [11]:
poc_sum=5
Dim=BlockDataStore.shape
count0=0
count1=0
count2=0
count3=0
count4=0
for rowx in range(Dim[0]):
    if(BlockDataStore[rowx][0]==0):
        count0=count0+1
    elif(BlockDataStore[rowx][0]==1):
        count1=count1+1
    elif(BlockDataStore[rowx][0]==2):
        count2=count2+1
    elif(BlockDataStore[rowx][0]==3):
        count3=count3+1
    elif(BlockDataStore[rowx][0]==4):
        count4=count4+1

In [12]:
count_List=[]
count_List.append(count0)
count_List.append(count1)
count_List.append(count2)
count_List.append(count3)
count_List.append(count4)

In [13]:
max(count_List)
count_List

[2315, 2344, 2491, 2446, 2477]

In [14]:
poc_sum=5
Dim=BlockDataStore.shape
count0=0
count1=0
count2=0
count3=0
count4=0
count_temp=0
BlockDataStoreinFrame=np.zeros([max(count_List),6,5])
for index in range(len(count_List)):
    print(BlockDataStoreinFrame[0:count_List[index],:,index].shape)
    BlockDataStoreinFrame[0:count_List[index],:,index]=BlockDataStore[count_temp:(count_temp+count_List[index]),:]
    count_temp=count_temp+count_List[index]
    

(2315, 6)
(2344, 6)
(2491, 6)
(2446, 6)
(2477, 6)


In [15]:
BlockDataStoreinFrame[0:100,:,4]

array([[4.00000e+00, 0.00000e+00, 0.00000e+00, 3.20000e+01, 3.20000e+01,
        1.10000e+02],
       [4.00000e+00, 3.20000e+01, 0.00000e+00, 1.60000e+01, 3.20000e+01,
        1.13000e+03],
       [4.00000e+00, 4.80000e+01, 0.00000e+00, 1.60000e+01, 3.20000e+01,
        1.13000e+03],
       [4.00000e+00, 0.00000e+00, 3.20000e+01, 3.20000e+01, 4.00000e+00,
        1.14000e+03],
       [4.00000e+00, 0.00000e+00, 3.60000e+01, 3.20000e+01, 4.00000e+00,
        1.14000e+03],
       [4.00000e+00, 0.00000e+00, 4.00000e+01, 3.20000e+01, 1.60000e+01,
        1.14000e+03],
       [4.00000e+00, 0.00000e+00, 5.60000e+01, 3.20000e+01, 8.00000e+00,
        1.14000e+03],
       [4.00000e+00, 3.20000e+01, 3.20000e+01, 1.60000e+01, 1.60000e+01,
        1.11000e+03],
       [4.00000e+00, 4.80000e+01, 3.20000e+01, 1.60000e+01, 8.00000e+00,
        1.11200e+04],
       [4.00000e+00, 4.80000e+01, 4.00000e+01, 1.60000e+01, 4.00000e+00,
        1.11210e+05],
       [4.00000e+00, 4.80000e+01, 4.40000e+01, 1.6

In [16]:
BlockDataStoreinFrame.shape
Dim
BlockDataStoreinFrame[0,0,0]

0.0

In [17]:

Dim3=BlockDataStoreinFrame.shape
for ii in range(0,poc_sum):
    wb = Workbook()
    wb.save('./GroundTruthoutput'+str(ii)+'.xlsx')
# for rowx in range(Dim[0]):
#     poc_ind_orig=BlockDataStore[rowx][0]
#     row_xlsx=int(BlockDataStore[rowx][1]+1)
#     col_xlsx=int(BlockDataStore[rowx][2]+1)
#     row_xlsx_sum=int(BlockDataStore[rowx][3])
#     col_xlsx_sum=int(BlockDataStore[rowx][4])
for poc_index in range(Dim3[-1]):
    filepath_write = './GroundTruthoutput'+str(int(poc_index))+'.xlsx'
    wb = load_workbook(filepath_write)
    ws1 = ws1 = wb.create_sheet(title="Sheet")
    row_ind=1
    col_ind=1
    for rowx in range(Dim3[0]):
        row_xlsx=int(BlockDataStoreinFrame[rowx,1,poc_index]+1)
        col_xlsx=int(BlockDataStoreinFrame[rowx,2,poc_index]+1)
        row_xlsx_sum=int(BlockDataStoreinFrame[rowx,3,poc_index])
        col_xlsx_sum=int(BlockDataStoreinFrame[rowx,4,poc_index])

        for row_index in range(row_xlsx,row_xlsx+row_xlsx_sum):
            for col_index in range(col_xlsx,col_xlsx+col_xlsx_sum):
                ws1.cell(row=row_index,column=col_index).value=BlockDataStoreinFrame[rowx,-1,poc_index]
    wb.save(filepath_write)
                   



In [28]:
# ###总深度决定一共有几位，qt深度决定前几位为1，剩余的位数由bt和mt决定，统一按照先bt后mt的划分顺序，先水平后垂直
# class GroundTruth(Enum):
#     Nothing=0
#     Qt=1
#     Bt_horizon=2
#     Bt_vertical=3
#     Tt_horizon=4
#     Tt_Vertical=5

# Dim=BlockDataStore.shape

# Depth_max=7
# Depth_index=5
# Qt_Depth_index=6
# Bt_Depth_index=7
# Mt_Depth_index=8
# SliptSeries=(-1)*np.ones([1,Depth_max])#深度最多为7
# for rowx in range(Dim[0]):
#     SliptSeries[rowx][0:int(BlockDataStore[rowx][Qt_Depth_index])]=GroundTruth.Qt
#     if(BlockDataStore[rowx][Bt_Depth_index]==BlockDataStore[rowx][Mt_Depth_index]):
#         ##如果相等证明没有三叉树划分
#         SliptSeries[rowx][0:int(BlockDataStore[rowx][Bt_Depth_index]))]=GroundTruth.Bt_horizon


    


In [106]:
len(BlockDatastore[:][0])

63168

In [26]:
filepath_write = './Depthoutput.xlsx'
wb = load_workbook(filepath_write)
ws1 = wb['Sheet1']
row_ind=1
col_ind=1
for Depth in Depth_list:
    ws1.cell(row=row_ind,column=col_ind).value=Depth
    row_ind=row_ind+1
wb.save(filepath_write)


